In [12]:
# Carregamento das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ast 
from difflib import get_close_matches

In [13]:
df_idade = pd.read_csv('./Dataset/idade_tratado.csv', sep=';', encoding='utf-8')

In [14]:
import pandas as pd

df_municipio = pd.read_csv(
    './Dataset/municipios_br.csv',
    sep=',',              # separador vírgula
    quotechar='"',        # trata campos entre aspas corretamente
    engine='python',      # interpretador mais tolerante
    encoding='utf-8',
    on_bad_lines='skip'   # se houver linhas corrompidas, ignora
)

print(df_municipio.columns.tolist())

['id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf', 'id_municipio_bcb', 'nome', 'capital_uf', 'id_comarca', 'id_regiao_saude', 'nome_regiao_saude', 'id_regiao_imediata', 'nome_regiao_imediata', 'id_regiao_intermediaria', 'nome_regiao_intermediaria', 'id_microrregiao', 'nome_microrregiao', 'id_mesorregiao', 'nome_mesorregiao', 'id_regiao_metropolitana', 'nome_regiao_metropolitana', 'ddd', 'id_uf', 'sigla_uf', 'nome_uf', 'nome_regiao', 'amazonia_legal', 'centroide']


In [15]:
df_idade.columns

Index(['MUNICIPIO', '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
       '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos',
       '60 a 69 anos', '70 a 79 anos', '80 anos e mais', 'Total'],
      dtype='object')

In [16]:
df_municipio.columns

Index(['id_municipio', 'id_municipio_6', 'id_municipio_tse', 'id_municipio_rf',
       'id_municipio_bcb', 'nome', 'capital_uf', 'id_comarca',
       'id_regiao_saude', 'nome_regiao_saude', 'id_regiao_imediata',
       'nome_regiao_imediata', 'id_regiao_intermediaria',
       'nome_regiao_intermediaria', 'id_microrregiao', 'nome_microrregiao',
       'id_mesorregiao', 'nome_mesorregiao', 'id_regiao_metropolitana',
       'nome_regiao_metropolitana', 'ddd', 'id_uf', 'sigla_uf', 'nome_uf',
       'nome_regiao', 'amazonia_legal', 'centroide'],
      dtype='object')

In [17]:
with open('./Dataset/municipios_br.csv', encoding='utf-8') as f:
    for i in range(2):
        print(f.readline())

id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,nome,capital_uf,id_comarca,id_regiao_saude,nome_regiao_saude,id_regiao_imediata,nome_regiao_imediata,id_regiao_intermediaria,nome_regiao_intermediaria,id_microrregiao,nome_microrregiao,id_mesorregiao,nome_mesorregiao,id_regiao_metropolitana,nome_regiao_metropolitana,ddd,id_uf,sigla_uf,nome_uf,nome_regiao,amazonia_legal,centroide

5101837,510183,,,,Boa Esperança do Norte,,,,,510008,Sorriso,5103,Sinop,,,,,,,,51,MT,Mato Grosso,Centro-Oeste,1,POINT(-54.9417429754238 -13.463197935201)



In [18]:
if 'centroide' not in df_municipio.columns:
    print("⚠️ coluna 'centroide' não encontrada. Refaça leitura do arquivo com separador correto.")

In [20]:
def encontrar_municipio(nome_idade, lista_municipios):
    """
    Encontra o município mais próximo na lista
    """
    matches = get_close_matches(nome_idade, lista_municipios, n=1, cutoff=0.8)
    return matches[0] if matches else None

# Criar dicionário de mapeamento
mapa_municipios = {}
lista_municipios = df_municipio['nome'].str.upper().tolist()

for municipio in df_idade['MUNICIPIO'].unique():
    municipio_limpo = municipio.upper().strip()
    match = encontrar_municipio(municipio_limpo, lista_municipios)
    if match:
        mapa_municipios[municipio] = match

# Aplicar o mapeamento
df_idade['municipio_mapeado'] = df_idade['MUNICIPIO'].map(mapa_municipios)

# Fazer o merge
df_merged = df_idade.merge(
    df_municipio,
    left_on='municipio_mapeado',
    right_on=df_municipio['nome'].str.upper(),
    how='left'
)


In [ ]:
# Alternativa usando shapely (se disponível)
try:
    from shapely import wkt
    
    def extrair_coords_wkt(centroide):
        try:
            if pd.isna(centroide):
                return pd.Series({'longitude': None, 'latitude': None})
            point = wkt.loads(str(centroide))
            return pd.Series({'longitude': point.x, 'latitude': point.y})
        except:
            return pd.Series({'longitude': None, 'latitude': None})
    
    coordenadas = df_merged['centroide'].apply(extrair_coords_wkt)
    df_merged = pd.concat([df_merged, coordenadas], axis=1)
except ImportError:
    print("Biblioteca 'shapely' não está instalada. As coordenadas não serão extraídas.")

In [ ]:
# Selecionar e organizar as colunas finais
colunas_idade = ['0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
                    '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos',
                    '60 a 69 anos', '70 a 79 anos', '80 anos e mais', 'Total']

df_final = df_merged[['MUNICIPIO', 'sigla_uf'] + colunas_idade + ['longitude', 'latitude']].copy()

# Salvar o arquivo
df_final.to_csv('municipios_idade_coordenadas.csv', index=False, encoding='utf-8-sig')

print(f"Arquivo salvo com sucesso!")
print(f"Total de registros: {len(df_final)}")
print(f"Com coordenadas: {df_final['latitude'].notna().sum()}")

✅ Arquivo salvo com sucesso!
Total de registros: 6207
Com coordenadas: 6139
